In [3]:
import gym
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

env = gym.make("BipedalWalker-v2")

DeprecatedEnv: Env BipedalWalker-v2 not found (valid versions include ['BipedalWalker-v3'])

In [ ]:
!pip install Box2D-kengz
!pip install git+https://github.com/pybox2d/pybox2d